In [13]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

import concurrent.futures

In [5]:

def get_rotten_tomatoes_rating(movie_name):

    url = f'https://www.rottentomatoes.com/m/{movie_name.replace(" ", "_")}'
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    thumbnail_scoreboard_wrap = soup.find('div', class_='thumbnail-scoreboard-wrap')
    rating_element = thumbnail_scoreboard_wrap.find('score-board') if thumbnail_scoreboard_wrap else None

    if rating_element:
        rating = rating_element.get('tomatometerscore', 'NaN')
    else:
        rating = 'NaN'

    return rating

In [9]:
data = pd.read_csv('netflix_titles.csv')

movie_names = data.title.to_list()

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

    ratings = list(executor.map(get_rotten_tomatoes_rating, movie_names))

df_rate = pd.DataFrame({'title': movie_names, 'rt_rating': ratings}) 

data = data.merge(df_rate, on='title', how='inner')

data.to_excel('netflix_data_with_rating.xlsx', engine='xlsxwriter')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
